## Introduction to Laplacian Based Feature Selection for Unlabeled Dataset
### Author - Huzaifa Kapasi, Sr. Architect, FullStack Datascientist

#### Note - Read the article first to understand the implied concepts behind Laplacian Score.


In [8]:
import scipy.io
from scipy.sparse import *
import numpy as np


In [26]:
class feature_selector_laplace():
    
    def __init__(self, filename):
        
        self.filename =filename
        
    def load_data(self):
        
        mat = scipy.io.loadmat(self.filename)
        self.X=mat['X']
        self.Y= mat['Y']
        
        return self.X,self.Y
        
        

    def laplacian_score(self,X,W):
            
            
            """ Input - X: input data        
            W: input affinity matrix
            Output -score: laplacian score for each feature
            Reference- He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
                       Li, Jundong - ACM Computing Surveys (CSUR) Journal
            """
            # build the diagonal D matrix from affinity matrix W
            D = np.array(W.sum(axis=1))
            L = W
            tmp = np.dot(np.transpose(D), X)
            D = diags(np.transpose(D), [0])
            Xt = np.transpose(X)
            t1 = np.transpose(np.dot(Xt, D.todense()))
            t2 = np.transpose(np.dot(Xt, L.todense()))
    
            # compute the numerator of Lr
            D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    
            # compute the denominator of Lr
            L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    
            # avoid the denominator of Lr to be 0
            D_prime[D_prime < 1e-12] = 10000
    
            # compute laplacian score for all features
            score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]
    
            return np.transpose(score)
    
    
    def rank(self,score):
    
            idx = np.argsort(score, 0)
            return idx
        
    def affinity_W(self,X):
            """
            Construct the affinity matrix W 
            Input - X: input data
            Output -W: output affinity matrix W
            """
            print('affinity_W()')
            n_samples, n_features = np.shape(X)
            
            
            k = 20       
            #numpy.savetxt("test.csv", X, delimiter=",")
            # normalize the data first
            X_norm = np.power(np.sum(X*X, axis=1), 0.5)
           
            for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_norm[i])
           
            # compute pairwise cosine distances
            # Cosine distance is dot product 
            D_cosine = np.dot(X, np.transpose(X))
            
            # sort the distance matrix D in ascending order
            dump = np.sort(-D_cosine, axis=1)
            idx = np.argsort(-D_cosine, axis=1)
            idx_new = idx[:, 0:k+1]
            dump_new = -dump[:, 0:k+1]
            G = np.zeros((n_samples*(k+1), 3))
            G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
            G[:, 1] = np.ravel(idx_new, order='F')
            G[:, 2] = np.ravel(dump_new, order='F')
            
            # build the sparse affinity matrix W
    
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W
    

In [27]:
path = r"C:\Users\huzaifa_kapasi\Documents\inter\Laplace_Feature\COIL20.mat"

fsl=feature_selector_laplace(path)
X,Y=fsl.load_data()


W = fsl.affinity_W(X)
score = fsl.laplacian_score(X,W)
score_ranked = fsl.rank(score)


affinity_W()
[[27.16380936 27.16675861 27.17627594 ... 28.16380936 28.16380936
  28.16380936]
 [29.8394775  29.83652825 29.84196036 ... 30.83652825 30.83652825
  30.83652825]
 [34.16281382 34.15577935 34.15034724 ... 35.15034724 35.15034724
  35.15034724]
 ...
 [20.05788142 20.05788142 20.05788142 ... 19.05788142 19.07404329
  19.08730127]
 [20.07769454 20.07769454 20.07769454 ... 19.09385641 19.07769454
  19.09947886]
 [20.0738987  20.0738987  20.0738987  ... 19.10331854 19.09568302
  19.0738987 ]]


AttributeError: 'matrix' object has no attribute 'todense'

In [23]:
print(score_ranked)

[ 510  478  542  445  477  446  509  413  476  444  541  574  412  482
  514  508  573  867  546  381  450  449  540  380  414  418  513  835
  174  838  802  605  481  419  578  387 1001  451  417  206  572  474
  606  770  511  386  806  483  579  969  198  442  604  547  545  968
  834  475  637  506  515  936 1002  443  543  538  737   10  411  480
  899  230  738  173  217  410  512  216  135    9  611  610  507  804
  420  705  970  184  741  348  249  707  706  636  846  379  845  774
  142  709  355  674  388  814   41  207  539  141  175  772  937  903
  378  877  935  577  385  739  642  773  803  349  870  479  143  901
  108   75  354  908  813  166  964  167  452   76  570  422  571  390
  109  742  932   72  580  836  389   42  347  152  940  740  997  638
  131  668  938  456  205  609  151  871  669  185   73  939  782  421
  900  868  448  409   43  805  328  356  673  544  112  327  603   40
  424  441  454  775  488  183  548  382  602  520  771  837  971  876
  248 

In [ ]:
def laplacian_score(self,X,W):
            
            
    """ Input - X: input data        
    W: input affinity matrix
    Output -score: laplacian score for each feature
    Reference- He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
               Li, Jundong - ACM Computing Surveys (CSUR) Journal
    """
    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    
    # compute laplacian score for all features
    score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]
    
    return np.transpose(score)
 